In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import torchreid
from scipy.spatial.distance import cosine
from deep_sort_realtime.deepsort_tracker import DeepSort
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Initializing the models
yolo_model = YOLO("yolo11n.pt")
extractor = torchreid.utils.FeatureExtractor(
    model_name='osnet_x1_0',
    device='cuda'
)

# Initializing the DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3)

# Gallery to store unique person embeddings
gallery = {}
next_person_id = 0
person_id_map = {}

cap = cv2.VideoCapture("Raw/shop.mp4")
min_conf = 0.5
while True:
    detections = []
    check, frame = cap.read()
    if not check:
        break
    
    # YOLO detection
    result = yolo_model(frame, verbose=False)[0]
    for box in result.boxes:
        if box.cls[0] != 0 or box.conf[0] < min_conf:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)
    
    for track in tracks:
        if not track.is_confirmed():
            continue
        
        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        
        # Check if the track ID is in our person_id map
        if track_id not in person_id_map:
            # New track ID, perform ReID check
            cropped = frame[t:b, l:r]
            if cropped.size == 0:
                continue
            
            resized = cv2.resize(cropped, (128, 256))
            feature = extractor([resized])[0].cpu()

            best_match_id = None
            best_similarity_score = 0.0

            # Compare with gallery
            if gallery:
                gallery_embeddings = np.array([data['embedding'].numpy() for data in gallery.values()])
                similarity_scores = cosine_similarity(
                    feature.unsqueeze(0).numpy(),
                    gallery_embeddings
                )[0]
                
                max_score_idx = np.argmax(similarity_scores)
                best_similarity_score = similarity_scores[max_score_idx]
                
                # Minimum similarity threshold
                if best_similarity_score > 0.6:
                    best_match_id = list(gallery.keys())[max_score_idx]

            if best_match_id is not None:
                # Match found, link the new track_id to the existing person_id
                person_id_map[track_id] = best_match_id
                print(f"Track {track_id} matched with known person ID {best_match_id} (score={best_similarity_score:.2f})")
            else:
                # No match found, create a new person ID
                current_person_id = next_person_id
                next_person_id += 1
                
                gallery[current_person_id] = {'embedding': feature}
                person_id_map[track_id] = current_person_id
                print(f"Track {track_id} assigned new person ID {current_person_id}")

        # Get the person_id from the map
        person_id = person_id_map[track_id]
        
        # Draw
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f"Person {person_id}", (l, t - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("ReID + Tracking", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

/home/ritwik/Ritwik/Programming/ml/CCTV/deep-person-reid/torchreid/models/osnet.py:482: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)
/h

Successfully loaded imagenet pretrained weights from "/home/ritwik/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352
Track 1 assigned new person ID 0
Track 2 assigned new person ID 1
Track 3 matched with known person ID 0 (score=0.62)
Track 4 assigned new person ID 2
Track 5 matched with known person ID 1 (score=0.63)
Track 6 matched with known person ID 1 (score=0.65)
Track 7 matched with known person ID 1 (score=0.67)
Track 9 matched with known person ID 1 (score=0.70)
Track 11 matched with known person ID 2 (score=0.80)
Track 12 matched with known person ID 2 (score=0.71)
Track 13 matched with known person ID 1 (score=0.67)
Track 15 matched with known person ID 1 (score=0.65)
Track 21 matched with known person ID 0 (score=0.67)
Track 22 matched with known person ID 1 (score=0.62)
Track 25 matched with known per